# Solar Power Prediction - Regression Models

This notebook explores linear and quadratic regression models for predicting solar power output.

**Models:**
- Linear Regression with Feature Scaling
- Quadratic Regression (Quadratic in POA, Linear in Other Features)


In [ ]:

from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

DATASET_PATH = Path('../data/data-3/timeseries/61272.csv')

data = pd.read_csv(DATASET_PATH)

X, y = train_test_split(data, test_size=0.2, random_state=42)
X.sort_index(inplace=True)

X.drop(['time', 'direct_normal_irradiance', 'diffuse_radiation', 'wind_speed_80m', 'wind_speed_180m', 'weather_code'], axis=1, inplace=True)
X.head()

# corr_mat = X.corr()
# print(corr_mat)


## Linear Regression with Feature Scaling

Perform linear regression with 'power' as the output variable, applying StandardScaler to features.


In [ ]:
# Separate features and target variable
X_features = X.drop('power', axis=1)
y_target = X['power']

print("Features used for regression:")
print(list(X_features.columns))
print(f"\nNumber of samples: {len(X_features)}")
print(f"Number of features: {X_features.shape[1]}")


In [ ]:
# Apply feature scaling using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features)

# Convert back to DataFrame for easier interpretation
X_scaled_df = pd.DataFrame(X_scaled, columns=X_features.columns, index=X_features.index)

print("="*70)
print("FEATURE SCALING APPLIED")
print("="*70)
print("\nOriginal feature statistics:")
print(X_features.describe())
print("\n" + "="*70)
print("Scaled feature statistics (mean≈0, std≈1):")
print(X_scaled_df.describe())


In [ ]:
# Train linear regression model
lr_model = LinearRegression()
lr_model.fit(X_scaled, y_target)

# Make predictions
y_pred = lr_model.predict(X_scaled)

# Calculate performance metrics
mse = mean_squared_error(y_target, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_target, y_pred)
r2 = r2_score(y_target, y_pred)

print("="*70)
print("LINEAR REGRESSION RESULTS")
print("="*70)
print(f"\nModel Performance Metrics:")
print(f"  R² Score:                 {r2:.6f}")
print(f"  Mean Squared Error (MSE): {mse:.2f}")
print(f"  Root Mean Squared Error:  {rmse:.2f}")
print(f"  Mean Absolute Error:      {mae:.2f}")
print(f"\nModel Intercept: {lr_model.intercept_:.2f}")
print("\nFeature Coefficients:")

# Create DataFrame of coefficients
coef_df = pd.DataFrame({
    'Feature': X_features.columns,
    'Coefficient': lr_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print(coef_df.to_string(index=False))


In [ ]:
# Visualize actual vs predicted
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot: Actual vs Predicted
axes[0].scatter(y_target, y_pred, alpha=0.5, s=10)
axes[0].plot([y_target.min(), y_target.max()],
             [y_target.min(), y_target.max()],
             'r--', lw=2, label='Perfect prediction')
axes[0].set_xlabel('Actual Power', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Predicted Power', fontsize=12, fontweight='bold')
axes[0].set_title(f'Actual vs Predicted Power\n(R² = {r2:.4f})',
                  fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals plot
residuals = y_target - y_pred
axes[1].scatter(y_pred, residuals, alpha=0.5, s=10)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Power', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Residuals', fontsize=12, fontweight='bold')
axes[1].set_title('Residual Plot', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Feature importance visualization
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['green' if c > 0 else 'red' for c in coef_df['Coefficient']]
bars = ax.barh(coef_df['Feature'], coef_df['Coefficient'], color=colors, alpha=0.7, edgecolor='black')

ax.set_xlabel('Coefficient Value', fontsize=12, fontweight='bold')
ax.set_ylabel('Feature', fontsize=12, fontweight='bold')
ax.set_title('Feature Coefficients (Linear Regression)', fontsize=14, fontweight='bold')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()


## Quadratic Regression (Quadratic in POA, Linear in Other Features)

Add a quadratic term for plane_of_array_irradiance to capture non-linear effects while keeping other features linear.


In [ ]:
# Create quadratic feature for plane_of_array_irradiance
X_features_quad = X_features.copy()
X_features_quad['plane_of_array_irradiance_squared'] = X_features['plane_of_array_irradiance'] ** 2

print("Features for quadratic regression:")
print(list(X_features_quad.columns))
print(f"\nNumber of features: {X_features_quad.shape[1]}")


In [ ]:
# Apply feature scaling to the quadratic feature set
scaler_quad = StandardScaler()
X_scaled_quad = scaler_quad.fit_transform(X_features_quad)

# Convert to DataFrame
X_scaled_quad_df = pd.DataFrame(X_scaled_quad, columns=X_features_quad.columns, index=X_features_quad.index)

print("="*70)
print("QUADRATIC FEATURE SCALING APPLIED")
print("="*70)
print("\nScaled quadratic features (first 5 rows):")
print(X_scaled_quad_df.head())


In [ ]:
# Train quadratic regression model
lr_quad_model = LinearRegression()
lr_quad_model.fit(X_scaled_quad, y_target)

# Make predictions
y_pred_quad = lr_quad_model.predict(X_scaled_quad)

# Calculate performance metrics
mse_quad = mean_squared_error(y_target, y_pred_quad)
rmse_quad = np.sqrt(mse_quad)
mae_quad = mean_absolute_error(y_target, y_pred_quad)
r2_quad = r2_score(y_target, y_pred_quad)

print("="*70)
print("QUADRATIC REGRESSION RESULTS")
print("="*70)
print(f"\nModel Performance Metrics:")
print(f"  R² Score:                 {r2_quad:.6f}")
print(f"  Mean Squared Error (MSE): {mse_quad:.2f}")
print(f"  Root Mean Squared Error:  {rmse_quad:.2f}")
print(f"  Mean Absolute Error:      {mae_quad:.2f}")
print(f"\nModel Intercept: {lr_quad_model.intercept_:.2f}")
print("\nFeature Coefficients:")

# Create DataFrame of coefficients
coef_quad_df = pd.DataFrame({
    'Feature': X_features_quad.columns,
    'Coefficient': lr_quad_model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print(coef_quad_df.to_string(index=False))

print("\n" + "="*70)
print("COMPARISON: LINEAR vs QUADRATIC")
print("="*70)
print(f"Linear Model R²:    {r2:.6f}")
print(f"Quadratic Model R²: {r2_quad:.6f}")
print(f"R² Improvement:     {r2_quad - r2:.6f} ({((r2_quad - r2) / r2 * 100):.2f}%)")
print(f"\nLinear Model RMSE:    {rmse:.2f}")
print(f"Quadratic Model RMSE: {rmse_quad:.2f}")
print(f"RMSE Improvement:     {rmse - rmse_quad:.2f} ({((rmse - rmse_quad) / rmse * 100):.2f}%)")


In [ ]:
# Visualize quadratic model performance
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot: Actual vs Predicted (Quadratic)
axes[0].scatter(y_target, y_pred_quad, alpha=0.5, s=10, label='Quadratic model')
axes[0].plot([y_target.min(), y_target.max()],
             [y_target.min(), y_target.max()],
             'r--', lw=2, label='Perfect prediction')
axes[0].set_xlabel('Actual Power', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Predicted Power', fontsize=12, fontweight='bold')
axes[0].set_title(f'Quadratic Model: Actual vs Predicted Power\n(R² = {r2_quad:.4f})',
                  fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals plot (Quadratic)
residuals_quad = y_target - y_pred_quad
axes[1].scatter(y_pred_quad, residuals_quad, alpha=0.5, s=10)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Power', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Residuals', fontsize=12, fontweight='bold')
axes[1].set_title('Quadratic Model: Residual Plot', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Compare residuals between linear and quadratic models
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Linear model residuals
axes[0].scatter(y_pred, residuals, alpha=0.5, s=10, c='blue')
axes[0].axhline(y=0, color='r', linestyle='--', lw=2)
axes[0].set_xlabel('Predicted Power', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Residuals', fontsize=12, fontweight='bold')
axes[0].set_title(f'Linear Model Residuals\n(RMSE = {rmse:.2f})',
                  fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Quadratic model residuals
axes[1].scatter(y_pred_quad, residuals_quad, alpha=0.5, s=10, c='green')
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Power', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Residuals', fontsize=12, fontweight='bold')
axes[1].set_title(f'Quadratic Model Residuals\n(RMSE = {rmse_quad:.2f})',
                  fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Visualize the quadratic relationship with POA irradiance
# Sort by POA for cleaner visualization
poa_sorted_idx = X_features['plane_of_array_irradiance'].sort_values().index
poa_sorted = X_features.loc[poa_sorted_idx, 'plane_of_array_irradiance']
y_sorted = y_target.loc[poa_sorted_idx]

# Get predictions for sorted data
X_sorted_quad = X_features_quad.loc[poa_sorted_idx]
X_sorted_quad_scaled = scaler_quad.transform(X_sorted_quad)
y_pred_sorted_quad = lr_quad_model.predict(X_sorted_quad_scaled)

# Also get linear predictions for comparison
X_sorted_linear = X_features.loc[poa_sorted_idx]
X_sorted_linear_scaled = scaler.transform(X_sorted_linear)
y_pred_sorted_linear = lr_model.predict(X_sorted_linear_scaled)

fig, ax = plt.subplots(figsize=(12, 6))

ax.scatter(poa_sorted, y_sorted, alpha=0.3, s=10, label='Actual data', c='gray')
ax.plot(poa_sorted, y_pred_sorted_linear, 'b-', linewidth=2, alpha=0.7, label=f'Linear model (R²={r2:.4f})')
ax.plot(poa_sorted, y_pred_sorted_quad, 'g-', linewidth=2, alpha=0.7, label=f'Quadratic model (R²={r2_quad:.4f})')

ax.set_xlabel('Plane of Array Irradiance', fontsize=12, fontweight='bold')
ax.set_ylabel('Power', fontsize=12, fontweight='bold')
ax.set_title('Power vs POA Irradiance: Linear vs Quadratic Fit', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
